In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import requests
import re
import math
from collections import Counter

In [3]:
# 1. Data preparation
def get_auguste_maquet_corpus():
    url = "https://www.gutenberg.org/files/7849/7849-0.txt"
    response = requests.get(url)
    text = response.text
    # Remove header and footer
    start = text.find("*** START OF THE PROJECT GUTENBERG EBOOK")
    end = text.find("*** END OF THE PROJECT GUTENBERG EBOOK")
    text = text[start:end]
    # Clean text
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

# 2. Tokenization and Vocabulary
def tokenize(text):
    return text.split()

In [4]:
class Vocabulary:
    def __init__(self, tokens):
        self.itos = ["<unk>", "<pad>", "<sos>", "<eos>"] + list(set(tokens))
        self.stoi = {token: i for i, token in enumerate(self.itos)}
    
    def __len__(self):
        return len(self.itos)
    
    def encode(self, tokens):
        return [self.stoi.get(token, self.stoi["<unk>"]) for token in tokens]
    
    def decode(self, ids):
        return [self.itos[id] for id in ids]

In [5]:

corpus = get_auguste_maquet_corpus()
tokens = tokenize(corpus)
vocab = Vocabulary(tokens)

In [6]:
class TextDataset(Dataset):
    def __init__(self, text, vocab, seq_length):
        self.text = text
        self.vocab = vocab
        self.seq_length = seq_length
        self.tokens = tokenize(self.text)
        self.data = self.vocab.encode(self.tokens)

    def __len__(self):
        return len(self.data) - self.seq_length

    def __getitem__(self, idx):
        x = torch.tensor(self.data[idx:idx+self.seq_length])
        y = torch.tensor(self.data[idx+1:idx+self.seq_length+1])
        return x, y

In [21]:
# 3. Model architecture
class TransformerDecoder(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_layers, dim_feedforward):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers)
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, tgt, tgt_mask=None):
        tgt = self.embedding(tgt) * math.sqrt(self.embedding.embedding_dim)
        tgt = self.pos_encoder(tgt)
        output = self.transformer_decoder(tgt, tgt, tgt_mask)
        return self.fc_out(output)
    
    def calculate_loss(self, output, target):
        return self.criterion(output.view(-1, output.size(-1)), target.view(-1))
    
    
    

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0)]

In [28]:

# 4. Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seq_length = 64
batch_size = 32
d_model = 256
nhead = 8
num_layers = 4
dim_feedforward = 1024
lr = 0.001
epochs = 10

dataset = TextDataset(corpus, vocab, seq_length)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

model = TransformerDecoder(len(vocab), d_model, nhead, num_layers, dim_feedforward).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)
print(model)

TransformerDecoder(
  (embedding): Embedding(5108, 256)
  (pos_encoder): PositionalEncoding()
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-3): 4 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
        )
        (linear1): Linear(in_features=256, out_features=1024, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=1024, out_features=256, bias=True)
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(

In [23]:
from tqdm import tqdm
# 5. Training loop
def train():
    model.train()
    total_loss = 0
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        # print(x.shape)
        output = model(x)
        loss = criterion(output.view(-1, len(vocab)), y.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

for epoch in tqdm(range(epochs)):
    loss = train()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss:.4f}")

# 6. Text generation
def generate_text(model, start_sequence, max_length=100):
    model.eval()
    with torch.no_grad():
        current_sequence = start_sequence
        for _ in range(max_length):
            input_tensor = torch.tensor(vocab.encode(current_sequence)).unsqueeze(0).to(device)
            output = model(input_tensor)
            next_token_idx = output[0, -1, :].argmax().item()
            next_token = vocab.itos[next_token_idx]
            current_sequence.append(next_token)
            if next_token == "<eos>":
                break
    return " ".join(current_sequence)

 10%|█         | 1/10 [01:08<10:12, 68.06s/it]

Epoch 1/10, Loss: 6.1964


 20%|██        | 2/10 [02:16<09:04, 68.03s/it]

Epoch 2/10, Loss: 6.1701


 30%|███       | 3/10 [03:23<07:55, 67.96s/it]

...

Epoch 8/10, Loss: 6.1521


 90%|█████████ | 9/10 [10:15<01:08, 68.41s/it]

Epoch 9/10, Loss: 6.1494


100%|██████████| 10/10 [11:21<00:00, 68.15s/it]

Epoch 10/10, Loss: 6.1515


In [24]:
# Example usage
start_sequence = ["the", "count", "of"]
generated_text = generate_text(model, start_sequence)
print(generated_text)

the count of the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the


In [25]:
# Update the text generation function to calculate perplexity
def generate_text_with_perplexity(model, start_sequence, max_length=10):
    model.eval()
    generated_sequence = start_sequence.copy()
    total_log_likelihood = 0
    total_tokens = 0
    
    with torch.no_grad():
        for _ in range(max_length):
            input_tensor = torch.tensor(vocab.encode(generated_sequence)).unsqueeze(0).to(device)
            output = model(input_tensor)
            next_token_logits = output[0, -1, :]
            next_token_idx = next_token_logits.argmax().item()
            next_token = vocab.itos[next_token_idx]
            
            # Calculate log likelihood of the chosen token
            log_likelihood = -model.criterion(next_token_logits.unsqueeze(0), torch.tensor([next_token_idx]).to(device)).item()
            total_log_likelihood += log_likelihood
            total_tokens += 1
            
            generated_sequence.append(next_token)
            if next_token == "<eos>" or len(generated_sequence) >= max_length:
                break
    
    generated_text = " ".join(generated_sequence)
    perplexity = math.exp(-total_log_likelihood / total_tokens)
    return generated_text, perplexity

In [26]:
# Example usage
start_sequence = ["the", "count", "of"]
generated_text, text_perplexity = generate_text_with_perplexity(model, start_sequence)
print("Generated Text:")
print(generated_text)
print(f"Text Perplexity: {text_perplexity:.4f}")

Generated Text:
the count of the the the the the the the
Text Perplexity: 16.5952


## New

In [29]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seq_length = 128  # Increased sequence length
batch_size = 64   # Increased batch size
d_model = 512     # Increased model dimension
nhead = 8
num_layers = 6    # Increased number of layers
dim_feedforward = 2048
lr = 0.0001       # Decreased learning rate
epochs = 50       # Increased number of epochs

dataset = TextDataset(corpus, vocab, seq_length)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

model = TransformerDecoder(len(vocab), d_model, nhead, num_layers, dim_feedforward).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

In [31]:
# Gradient clipping
clip_value = 1.0

def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0
    total_tokens = 0
    for _, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x)
        loss = model.calculate_loss(output, y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
        optimizer.step()
        total_loss += loss.item() * y.numel()
        total_tokens += y.numel()
    
    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    return avg_loss, perplexity


def validate(model, dataloader, device):
    model.eval()
    total_loss = 0
    total_tokens = 0
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            output = model(x)
            loss = model.calculate_loss(output, y)
            total_loss += loss.item() * y.numel()
            total_tokens += y.numel()
    
    avg_loss = total_loss / total_tokens
    perplexity = math.exp(avg_loss)
    return avg_loss, perplexity


# Training loop
for epoch in tqdm(range(epochs), desc='Training'):
    train_loss, train_perplexity = train_epoch(model, train_dataloader, optimizer, device)
    val_loss, val_perplexity = validate(model, val_dataloader, device)
    scheduler.step()
    
    print(f"Epoch {epoch+1}/{epochs}")
    print(f"Train Loss: {train_loss:.4f}, Train Perplexity: {train_perplexity:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Perplexity: {val_perplexity:.4f}")
    print("--------------------")

Training:   2%|▏         | 1/50 [03:36<2:56:50, 216.53s/it]

Epoch 1/50
Train Loss: 6.1937, Train Perplexity: 489.6367
Val Loss: 6.1833, Val Perplexity: 484.5922
--------------------


Training:   4%|▍         | 2/50 [07:14<2:54:03, 217.57s/it]

Epoch 2/50
Train Loss: 6.1852, Train Perplexity: 485.5127
Val Loss: 6.1799, Val Perplexity: 482.9481
--------------------


Training:   6%|▌         | 3/50 [10:52<2:50:36, 217.80s/it]

...

Epoch 48/50
Train Loss: 6.1610, Train Perplexity: 473.8829
Val Loss: 6.1604, Val Perplexity: 473.6268
--------------------


Training:  98%|█████████▊| 49/50 [2:57:54<03:37, 217.41s/it]

Epoch 49/50
Train Loss: 6.1610, Train Perplexity: 473.9024
Val Loss: 6.1604, Val Perplexity: 473.6267
--------------------


Training: 100%|██████████| 50/50 [3:01:31<00:00, 217.83s/it]

Epoch 50/50
Train Loss: 6.1609, Train Perplexity: 473.8779
Val Loss: 6.1604, Val Perplexity: 473.6267
--------------------


In [32]:
# Improved text generation function
def generate_text_with_sampling(model, start_sequence, max_length=100, temperature=0.7, top_k=50, top_p=0.9):
    model.eval()
    generated_sequence = start_sequence.copy()
    
    with torch.no_grad():
        for _ in range(max_length):
            input_tensor = torch.tensor(vocab.encode(generated_sequence)).unsqueeze(0).to(device)
            output = model(input_tensor)
            next_token_logits = output[0, -1, :] / temperature
            
            # Top-k sampling
            top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)
            
            # Top-p (nucleus) sampling
            sorted_logits, sorted_indices = torch.sort(top_k_logits, descending=True)
            cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            indices_to_remove = sorted_indices[sorted_indices_to_remove]
            top_k_logits[indices_to_remove] = float('-inf')
            
            # Sample from the filtered distribution
            probs = torch.softmax(top_k_logits, dim=-1)
            next_token_idx = top_k_indices[torch.multinomial(probs, 1).item()].item()
            next_token = vocab.itos[next_token_idx]
            
            generated_sequence.append(next_token)
            if next_token == "<eos>" or len(generated_sequence) >= max_length:
                break
    
    return " ".join(generated_sequence)

# Example usage
start_sequence = ["the", "count", "of"]
generated_text = generate_text_with_sampling(model, start_sequence, temperature=0.7, top_k=50, top_p=0.9)
print("Generated Text:")
print(generated_text)

Generated Text:
the count of had the he its the that for i had the it but as but a to and you i to the the a and and he had to he to so to and and the and the a the that the the the he to was but and k to and the was his the this for you of but the at and was at the had and i the to and him and and not the of to to and this it the a the the that it him in the and the the and to


In [33]:
import torch
import torch.nn.functional as F
import math

# Improved text generation function with perplexity
def generate_text_with_sampling_and_perplexity(model, start_sequence, vocab, max_length=100, temperature=0.7, top_k=50, top_p=0.9):
    model.eval()
    generated_sequence = start_sequence.copy()
    perplexities = []
    
    with torch.no_grad():
        for _ in range(max_length):
            input_tensor = torch.tensor(vocab.encode(generated_sequence)).unsqueeze(0).to(device)
            output = model(input_tensor)
            next_token_logits = output[0, -1, :] / temperature
            
            # Top-k sampling
            top_k_logits, top_k_indices = torch.topk(next_token_logits, top_k)
            
            # Top-p (nucleus) sampling
            sorted_logits, sorted_indices = torch.sort(top_k_logits, descending=True)
            cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
            sorted_indices_to_remove = cumulative_probs > top_p
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            indices_to_remove = sorted_indices[sorted_indices_to_remove]
            top_k_logits[indices_to_remove] = float('-inf')
            
            # Sample from the filtered distribution
            probs = F.softmax(top_k_logits, dim=-1)
            next_token_idx = top_k_indices[torch.multinomial(probs, 1).item()].item()
            next_token = vocab.itos[next_token_idx]
            
            # Calculate perplexity for the generated token
            token_prob = probs[top_k_indices == next_token_idx].item()  # Probability of the sampled token
            perplexity = math.exp(-math.log(token_prob)) if token_prob > 0 else float('inf')
            perplexities.append(perplexity)
            
            generated_sequence.append(next_token)
            print(f"Generated Token: {next_token}, Perplexity: {perplexity:.4f}")
            
            # Break if end-of-sequence token is generated or max length is reached
            if next_token == "<eos>" or len(generated_sequence) >= max_length:
                break
    
    return " ".join(generated_sequence), perplexities

# Example usage
start_sequence = ["the", "count", "of"]
generated_text, perplexities = generate_text_with_sampling_and_perplexity(model, start_sequence, vocab, temperature=0.7, top_k=50, top_p=0.9)

print("Generated Text:")
print(generated_text)
print("Perplexities:", perplexities)

Generated Token: and, Perplexity: 17.1910
Generated Token: to, Perplexity: 8.6642
Generated Token: said, Perplexity: 64.9131
Generated Token: you, Perplexity: 35.3226
Generated Token: the, Perplexity: 5.7980
Generated Token: i, Perplexity: 50.8094
Generated Token: the, Perplexity: 4.3551
Generated Token: it, Perplexity: 27.9179
Generated Token: and, Perplexity: 14.5621
Generated Token: the, Perplexity: 5.3899
Generated Token: with, Perplexity: 75.6919
Generated Token: was, Perplexity: 43.3846
Generated Token: he, Perplexity: 17.4259
Generated Token: at, Perplexity: 122.5639
Generated Token: it, Perplexity: 24.7624
Generated Token: to, Perplexity: 7.1765
Generated Token: on, Perplexity: 98.8945
Generated Token: and, Perplexity: 17.3603
Generated Token: to, Perplexity: 8.6642
Generated Token: of, Perplexity: 18.0934
Generated Token: of, Perplexity: 22.5940
Generated Token: the, Perplexity: 3.9198
Generated Token: at, Perplexity: 81.3594
Generated Token: k, Perplexity: 32.5852
Generated T